### Make sure to run in the ATLAS controls ACS023 terminal via controlsys login: python3.6 api_server.py

## Start of good code to run loops to collect data

In [5]:
#settings / loading
import requests
import pandas as pd
import os
from random import random
import time
write2database = True #write to influx?
post_settings = False #whether to send new magnet settings
setting_list = ["QTA800:CONTROL_Y1","QTA800:CONTROL_X1","QTA800:CONTROL_Y2",
                "STA801:CONTROL_X","STA801:CONTROL_Y",
                "QDA800:CONTROL_X","QDA800:CONTROL_Y"]#,
                # "MGA002:CONTROL",
                # "QDA002:CONTROL_X","QDA002:CONTROL_Y",
                # "STA004:CONTROL_X","STA004:CONTROL_Y",
                # "QDA001:CONTROL_X","QDA001:CONTROL_Y",
                # "MGA001:CONTROL",
                # "QTA001:CONTROL_X1","QTA001:CONTROL_X2","QTA001:CONTROL_Y",
                # "STA003:CONTROL_X","STA003:CONTROL_Y",
                # "STA002:CONTROL_X","STA002:CONTROL_Y",
                # "STA001:CONTROL_X","STA001:CONTROL_Y"] #which magnets to read
#setting_list = ["R501:PHASEB","R501:CONTROL_AMP"]
max_vals = [10.0,15.0,20.0,1.5,1.5] #max field setting for bl magnets
min_vals = [0.5,0.5,0.5,-1.5,-1.5]
off_vals = [0.0,0.0,0.0,-1.1,-1.1]
delta_vals = [0.5,0.5,0.5,0.02,0.02] #amount by which to change 
orig_vals = [4.051278,5.904756,5.272509,-1.036212,0.827078] #initial tune settings before scans
num_loops = 10000 #finite numer of samples if using
num_samples = 2 #times to measure the quads before changing them # not used at present

In [6]:
#initial read and need to save in df idealy
for loops in range(1000):
    print("On Loop Number: ",loops)
    i=0
    for names in setting_list:
    #print(names)
        #read present setting
        if (write2database):
            #for names in setting_list:
            #read new settings
            resp = requests.get('http://146.139.66.23:5000/', json={
            "setting": [names],
            "fcup": [],
            "bpm": [] })
            df = pd.json_normalize(resp.json())
            temp_cntrl_name = "setting.control." + names
            temp_mon_name = "setting.monitor." + names
            # print("Monitor: {}: {:.3f} [{:.3f}]".format(temp_mon_name,df[temp_mon_name].iat[0],df[temp_cntrl_name].iat[0]))
            dbwrite = "influx -execute \'insert bl,tag={} value={}\' -database=db".format(temp_cntrl_name,df[temp_cntrl_name].iat[0])
            os.system(dbwrite)
            dbwrite = "influx -execute \'insert bl,tag={} value={}\' -database=db".format(temp_mon_name,df[temp_mon_name].iat[0])
            os.system(dbwrite)
    time.sleep(1) #time to see detector rates

On Loop Number:  0
On Loop Number:  1
On Loop Number:  2
On Loop Number:  3
On Loop Number:  4
On Loop Number:  5
On Loop Number:  6
On Loop Number:  7
On Loop Number:  8
On Loop Number:  9
On Loop Number:  10
On Loop Number:  11
On Loop Number:  12
On Loop Number:  13
On Loop Number:  14
On Loop Number:  15
On Loop Number:  16
On Loop Number:  17
On Loop Number:  18
On Loop Number:  19
On Loop Number:  20
On Loop Number:  21
On Loop Number:  22
On Loop Number:  23
On Loop Number:  24
On Loop Number:  25
On Loop Number:  26
On Loop Number:  27
On Loop Number:  28
On Loop Number:  29
On Loop Number:  30
On Loop Number:  31
On Loop Number:  32
On Loop Number:  33
On Loop Number:  34
On Loop Number:  35
On Loop Number:  36
On Loop Number:  37
On Loop Number:  38
On Loop Number:  39
On Loop Number:  40
On Loop Number:  41
On Loop Number:  42
On Loop Number:  43
On Loop Number:  44
On Loop Number:  45
On Loop Number:  46
On Loop Number:  47
On Loop Number:  48
On Loop Number:  49
On Loop Nu

KeyboardInterrupt: 